In [1]:
import numpy as np
import pandas as pd  
import seaborn as sns 
import matplotlib.pyplot as plt  
import pandas.api.types
import sklearn.metrics  
import nibabel as nib
import os
import pydicom
from glob import glob
from tqdm import tqdm, trange
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToPILImage, ToTensor
from timm import create_model
import torch.nn as nn
import torch.optim as optim

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection"

In [3]:
Target_cols = ["bowel_healthy", "bowel_injury", "extravasation_healthy",
                   "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high","any_injury"]

In [4]:
file_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection"
train_csv=f"{file_path}/train.csv" 
train=pd.read_csv(train_csv) 

In [5]:
organs_healthy = [ 
          'bowel_healthy',
          'extravasation_healthy',
          'kidney_healthy',
          'liver_healthy',
          'spleen_healthy'
] 

In [6]:
low_high = [
            'bowel_injury',
            'extravasation_injury',
            'kidney_low',
            'kidney_high',
            'liver_high',
            'liver_low' , 
            'spleen_low',
            'spleen_high',
            'any_injury'
] 

In [7]:
train_series_meta = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')

In [8]:
test_series_meta=pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/test_series_meta.csv") 

In [9]:
image_labels = pd.read_csv( "/kaggle/input/rsna-2023-abdominal-trauma-detection/image_level_labels.csv")

In [10]:
BASE_DIR = "/kaggle/input/rsna-2023-abdominal-trauma-detection"
TRAIN_CSV_PATH = os.path.join(BASE_DIR, "train.csv")
TRAIN_IMAGES_PATH = os.path.join(BASE_DIR, "train_images")
TEST_IMAGES_PATH = os.path.join(BASE_DIR, "test_images")
SAMPLE_SUBMISSION_PATH = os.path.join(BASE_DIR, "sample_submission.csv")

In [11]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
sample_submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [12]:
test_df = sample_submission[['patient_id']]

In [13]:

def get_dicom_paths(patient_id, is_train=True):
    folder = TRAIN_IMAGES_PATH if is_train else TEST_IMAGES_PATH
    paths = []
    for dirpath, _, filenames in os.walk(os.path.join(folder, patient_id)):
        for file in filenames:
            paths.append(os.path.join(dirpath, file))
    return paths

In [14]:

transform = transforms.Compose([
    transforms.ToTensor(),
])

In [15]:
class CTDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, is_train=True):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.is_train = is_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        patient_id = str(self.dataframe.iloc[idx, 0])
        dicom_paths = get_dicom_paths(patient_id, self.is_train)
        if not dicom_paths:
            raise FileNotFoundError(f"No DICOM images found for patient {patient_id}")
        image = pydicom.dcmread(dicom_paths[0]).pixel_array
        image = image.astype(np.float32) / 65535.0  
        image = np.stack((image,) * 3, axis=-1)
        image *= 255
        image = image.astype(np.uint8)
        to_pil = ToPILImage()
        image = to_pil(image)
        resize_transform = Resize((512, 512))
        image = resize_transform(image)
        to_tensor = ToTensor()
        image = to_tensor(image)
        labels = self.dataframe.iloc[idx, 1:].values
        labels = torch.tensor(labels, dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, labels

In [16]:
batch_size = 8
train_dataset = CTDataset(dataframe=train_df, root_dir=TRAIN_IMAGES_PATH, is_train=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [17]:
test_dataset = CTDataset(dataframe=test_df, root_dir=TEST_IMAGES_PATH, is_train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
model = create_model('efficientnet_b4', pretrained=False)

In [19]:
torch.save(model.state_dict(), 'efficientnet_b4_weights.pth')

In [20]:
weights_path = 'efficientnet_b4_weights.pth'

def build_model(num_classes):
    model = create_model('efficientnet_b4', pretrained=False)
    
    if os.path.exists(weights_path):
        model.load_state_dict(torch.load(weights_path, map_location='cpu'), strict=False)  
    else:
        print(f'Warning: Weights file not found in path {weights_path}, training from scratch.')
    
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    
    return model

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_model(len(train_df.columns) - 1).to(device)

In [22]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        print(i)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
model.eval()
predictions = []

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = torch.sigmoid(model(images))
        predictions.append(outputs.cpu().numpy())

In [25]:
predictions = np.vstack(predictions)

In [26]:
predictions = predictions[:, :-1]

In [27]:
submission_df = pd.DataFrame(sample_submission['patient_id'], columns=['patient_id'])
predictions_df = pd.DataFrame(predictions, columns=sample_submission.columns[1:])
submission_df = pd.concat([submission_df, predictions_df], axis=1)

In [28]:
submission_filename = "submission.csv"
submission_df.to_csv(submission_filename, index=False)